In [36]:
# libraries
import numpy as np
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from datasets import Dataset

import sys

sys.path.append("../scripts")
import functions as f

.csv with all manual labels applied

In [ ]:
# Load the file
reddit = joblib.load('./data/pickle/reddit_labelled-sample.pkl')

In [38]:
# Apply preprocessing
reddit["processed_text"] = reddit["text"].apply(f.token_and_lemmatize_rob)

# Features and labels
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(reddit["processed_text"])  # Tfidf vectorization
y = reddit["label"]

In [39]:
# split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=13,
    stratify=y
)
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

Train and test RoBERTa (Robustly Optimized BERT)

In [40]:
# Convert sparse matrices to dense and prepare datasets
X_train_text = [" ".join(map(str, row)) for row in X_train.toarray()]
X_test_text = [" ".join(map(str, row)) for row in X_test.toarray()]

train_data = {"text": X_train_text, "labels": y_train.tolist()}
test_data = {"text": X_test_text, "labels": y_test.tolist()}

train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

In [41]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_function(examples):
    # Tokenize using Hugging Face tokenizer
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [42]:
train_dataset = train_dataset.map(f.tokenize_function, batched=True)
test_dataset = test_dataset.map(f.tokenize_function, batched=True)

Map:   0%|          | 0/1056 [00:00<?, ? examples/s]

Map:   0%|          | 0/265 [00:00<?, ? examples/s]

In [ ]:
# Load and train the model
roberta_model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base", num_labels=3
)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=roberta_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

# Evaluate the model
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)

# Decode predictions and calculate accuracy
decoded_preds = encoder.inverse_transform(preds)
decoded_labels = encoder.inverse_transform(y_test)

accuracy = accuracy_score(decoded_labels, decoded_preds)
print(f"RoBERTa model accuracy: {accuracy:.2f}")

# Classification report
print("Classification Report:")
print(
    classification_report(
        decoded_labels, decoded_preds, target_names=encoder.classes_, zero_division=0
    )
)

In [ ]:
# Define the save path
save_path = "./models"

# Save the trained model
roberta_model.save_pretrained(save_path)

# Save the tokenizer
tokenizer.save_pretrained(save_path)

# Save the LabelEncoder as a pickle object
joblib.dump(
    encoder,
    f"{save_path}/roberta_label_encoder.pkl",
)

['/Users/seshat/Documents/GitHub/labor_sentiment_analysis/models/roberta_label_encoder.pkl']

In [ ]:
# Load the model from the file
nb_classifier = joblib.load(
    "./models/custom_nb_model.pkl"
)

In [35]:
# Compare to SAME, UNALTERED custom Naive Bayes model from above applied to this test data subset.
# Extra testing!
# predict
y_pred = nb_classifier.predict(X_test)

# decode
custom_nb_pred = encoder.inverse_transform(y_pred)

y_test_decoded = encoder.inverse_transform(y_test)

accuracy = accuracy_score(y_test_decoded, custom_nb_pred)
print(f"Test set accuracy: {accuracy}")

# Detailed performance metrics
print("Classification Report:")
print(
    classification_report(
        y_test_decoded, custom_nb_pred, 
        target_names=encoder.classes_, 
        zero_division=0
    )
)

Test set accuracy: 0.43018867924528303
Classification Report:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        31
     neutral       0.47      0.67      0.55       134
    positive       0.33      0.24      0.28       100

    accuracy                           0.43       265
   macro avg       0.27      0.30      0.28       265
weighted avg       0.36      0.43      0.39       265



Compare, contrast and discuss these results